In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torchmetrics import F1Score
from torch.utils.data import Dataset, DataLoader

%matplotlib inline

In [3]:
df = pd.read_json('../df.json')